# San Francisco Crime Classification Challege

This is my approach for the [kaggle's San francisco Crime Classification challenge](https://www.kaggle.com/c/sf-crime)

## Reading our data

Firstly, let's read the data into a pandas dataframe:

In [2]:
import pandas as pd
training_data = pd.read_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/train 2.csv')
training_data.head(3)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414


Interestingly, It is a high dimentional dataset with 8049 row and 9 columns. We can also see the headers of the data frame as folows:

In [3]:
training_column_names = list(training_data.columns.values)
training_column_names

['Dates',
 'Category',
 'Descript',
 'DayOfWeek',
 'PdDistrict',
 'Resolution',
 'Address',
 'X',
 'Y']

Let's load the testing data:

In [4]:
testing_data = pd.read_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/test 2.csv')
testing_data.head(4)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [5]:
test_data_column_names  = list(testing_data.columns.values)
test_data_column_names

['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y']

This is how our data looks like (both match):

In [6]:
#create a third list with items from the training_column_names list which aren't 
#present in the test_data_column_names list. 
list(set(test_data_column_names) - set(training_column_names))

['Id']

In [7]:
#create a third list with items from the training_column_names list which aren't 
#present in the test_data_column_names list. 
list(set(training_column_names) - set(test_data_column_names))

['Descript', 'Category', 'Resolution']

In [8]:
training_data.head(1)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599


In [9]:
testing_data.head(1)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051


In [10]:
# id se va
# category es lo que se predice
# adress como texto
testing_data = testing_data.drop('Id', 1)
testing_data.head()

,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


# Feature engineering

## Training data

In [11]:
training_data.head(1)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599


In [12]:
#category_feature labels

category_feature = training_data['Category'].tolist()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
category_feature = le.fit_transform(category_feature)
category_feature = category_feature
#category_feature

In [13]:
from scipy import sparse
address_feature = training_data['Address'].tolist()
address_feature = le.fit_transform(address_feature)
address_feature = sparse.csr_matrix(address_feature)
print (address_feature.shape)
address_feature

(1, 878049)


<1x878049 sparse matrix of type '<class 'numpy.int64'>'
	with 878048 stored elements in Compressed Sparse Row format>

In [14]:
dayweek_feature = training_data['DayOfWeek'].tolist()
dayweek_feature = le.fit_transform(dayweek_feature)
dayweek_feature =sparse.csr_matrix(dayweek_feature)
print( dayweek_feature.shape)
dayweek_feature

(1, 878049)


<1x878049 sparse matrix of type '<class 'numpy.int64'>'
	with 744315 stored elements in Compressed Sparse Row format>

In [15]:
pddis_feature = training_data['PdDistrict'].tolist()
pddis_feature = le.fit_transform(pddis_feature)
pddis_feature = sparse.csr_matrix(pddis_feature)
pddis_feature = pddis_feature
print( pddis_feature.shape)
pddis_feature

(1, 878049)


<1x878049 sparse matrix of type '<class 'numpy.int64'>'
	with 788618 stored elements in Compressed Sparse Row format>

In [16]:
import numpy as np

X_combined_features = sparse.vstack([address_feature,dayweek_feature,pddis_feature]).T
y = category_feature
type(y)

numpy.ndarray

## print X_combined_features.toarray()


In [19]:
X_combined_features.shape
X_combined_features

<878049x3 sparse matrix of type '<class 'numpy.int64'>'
	with 2410981 stored elements in Compressed Sparse Column format>

# Dimentionality reduction

In [20]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_reduced = sel.fit_transform(X_combined_features)
X_reduced.shape

(878049, 3)

In [21]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X_new = SelectKBest(chi2, k=2).fit_transform(X_combined_features, y)
X_new.shape




(878049, 2)

## Testing data 

In [22]:
df2 = pd.read_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/test 2.csv').dropna()
df2.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [23]:
dayweek_training_feature = testing_data['DayOfWeek'].tolist()
dayweek_training_feature = le.fit_transform(dayweek_training_feature)
dayweek_training_feature =sparse.csr_matrix(dayweek_training_feature)
print (dayweek_training_feature.shape)
dayweek_training_feature

(1, 884262)


<1x884262 sparse matrix of type '<class 'numpy.int64'>'
	with 749559 stored elements in Compressed Sparse Row format>

DBSCAN over the coordinates

In [24]:
from sklearn.cluster import MiniBatchKMeans
db_scan = MiniBatchKMeans()
db_scan.partial_fit(df2[['X','Y']])

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=8,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [33]:
labels = db_scan.labels_[::10]
type(labels)
labels

array([4, 4, 6, ..., 3, 2, 4], dtype=int32)

In [4]:
import pickle as pickle
